# Mapreader Workshops 2024

----

First check you have the correct version of MapReader: v1.3.0

This can be downloaded from pypi using `pip install mapreader==1.3.0` or by checking out the repo at [this commit](https://github.com/Living-with-machines/MapReader/releases/tag/v1.3.0)

In [1]:
import mapreader
assert mapreader.__version__ == '1.3.0'

-------------

## Load

We will start by creating a new ``MapImages`` object by loading our 1000 pixel patches and adding our ``metadata.csv``.

To speed things up for the workshop, we will just load patches corresponding to ``map_75650907.png``.

In [ ]:
from mapreader import load_patches

In [ ]:
my_files = load_patches(patch_paths="./patches_1000_pixel/*75650907*png", parent_paths="./maps/map_75650907.png")

In [ ]:
my_files.add_metadata("./maps/metadata.csv", ignore_mismatch=True)

We then need to run the ``add_coord_increments()`` method, which calculates the change in latitude (dlat) and longitude (dlon) across each pixel.
This will be used to convert pixel coordinates to geo coordinates.

In [ ]:
my_files.add_coord_increments()

In [ ]:
parent_df, patch_df = my_files.convert_images()

In [ ]:
parent_df.head()

In [ ]:
patch_df.head()

## Detect and recognise text

Now we can load an already trained/fine-tuned text detection model and run the model inference on our patches. 

DeepSolo is a text detection and recognition framework so it produces bounding boxes, recognised text outputs and scores.

### Set up the DeepSoloRunner

In [ ]:
from mapreader import DeepSoloRunner

In [ ]:
# change these to your own paths, see the docs for more details on how to get these
cfg_file = "/Users/rwood/projects/DataCulture/ocr_test/detectron2_etc/DeepSolo/configs/R_50/IC15/finetune_150k_tt_mlt_13_15_textocr.yaml"
weights_file = "/Users/rwood/projects/DataCulture/ocr_test/detectron2_etc/ic15_res50_finetune_synth-tt-mlt-13-15-textocr.pth"

In [ ]:
my_runner = DeepSoloRunner(
    patch_df,
    parent_df,
    cfg_file = cfg_file,
    weights_file = weights_file,
)

### Run on all patches in the patch dataframe

In [ ]:
patch_predictions = my_runner.run_all(return_dataframe=True)

In [ ]:
patch_predictions.head()

We can visualise the outputs for each patch with the show method.

In [ ]:
my_runner.show(
    "patch-4000-2000-5000-3000-#map_75650907.png#.png",
    figsize=(15, 15),
    border_color="r",
    text_color="b",
    )

### Scale up to parent images

The ``convert_to_parent_pixel_bounds()`` method takes all the patch predictions and scales the up to the parent image.
This means all our bounding boxes are grouped at the parent-level, rather than at the patch-level.

In [ ]:
parent_predictions = my_runner.convert_to_parent_pixel_bounds(return_dataframe=True)

In [ ]:
parent_predictions.head()

Now we can visualise the outputs for each parent with the show method.

In [ ]:
my_runner.show(
    "map_75650907.png", 
    figsize=(15, 15),
    border_color="r",
    text_color="b",
)

### Convert pixel bounds to coordinates

Since we added our dlat and dlon (coordinate increments), we can convert out pixel coordinates to geo coordinates.
This will mean our bounding boxes are georeferenced.

In [ ]:
geo_predictions = my_runner.convert_to_coords(return_dataframe=True)

Saving these outputs will give you a geojson file you can load into a GIS software.

In [ ]:
my_runner.save_to_geojson("./deepsolo_outputs.geojson")